In [ ]:
import torch
import torch.nn as nn                   #for sequence api in torch
from torch.utils.data import DataLoader #for loading images
import numpy as np                      #just in case if you need numpy arrays
import torchvision.transforms as T      #Used for data preprocessing and converting images to tensors
import torchvision.datasets as dset
import torch.optim as optim             #For using the desired parameter update
import torch.nn.functional as F
from torchvision.datasets import ImageFolder
USE_GPU = True

if USE_GPU and torch.cuda.is_available():
  device = torch.device('cuda')
else:
  device = torch.device('cpu')

dtype = torch.float32
print("Using device: ",device)


In [ ]:
# !unzip "/content/drive/MyDrive/FER_Dataset.zip"

Data loading and pre-processing

In [ ]:

transform = T.Compose([T.RandomHorizontalFlip(),T.ToTensor()])
#Training
train_data = ImageFolder("/content/images/images/train",transform=transform)
loaded_train = DataLoader(train_data,batch_size=64,shuffle=True)
#Validation
validation_data = ImageFolder ("/content/images/images/validation",transform=transform)
loaded_validation = DataLoader(validation_data,batch_size=64,shuffle=False)

loss_history = []
validation_acc = []
training_acc = []

Visualizing the image

In [ ]:
import matplotlib.pyplot as plt
import random                   #For selecting random element from list

dataiter = iter(loaded_train)   #The iter() function in python represents the iterator similar to c++ iterators
images, labels = next(dataiter) #The next() method retrieves the object
expression = {0:"angry",1:"disgust",2:"fear",3:"happy",4:"neutral",5:"sad",6:"surprise"} #Create a dictionary for mapping accordingly
random_idx = random.sample(range(0,64),1)[0]     #Selects a random single number from 0-64
print("Target label: ",expression[int(labels[random_idx].numpy())])  #Converting it to numpy from tensor to fetch the label
plt.imshow(np.transpose(images[random_idx].numpy(), (1, 2, 0)))

Creating a method for predicting validation accuracy¶

In [ ]:
def check_accuracy_part(loader, model):
    print('Checking accuracy on validation set')
    num_correct = 0
    num_samples = 0
    model.eval()  # set model to evaluation mode
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)
            scores = model(x)
            _, preds = scores.max(1)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
        acc = float(num_correct) / num_samples
        print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))

Creating a method for predicting validation accuracy

In [ ]:
def check_accuracy_part(loader, model):
    print('Checking accuracy on validation set')
    num_correct = 0
    num_samples = 0
    model.eval()  # set model to evaluation mode
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)
            scores = model(x)
            _, preds = scores.max(1)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
        acc = float(num_correct) / num_samples
        print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))

Training the model

In [ ]:
def train_part(model, optimizer, epochs=1):
    """
    Train a model using the PyTorch Module API.

    Inputs:
    - model: A PyTorch Module giving the model to train.
    - optimizer: An Optimizer object we will use to train the model
    - epochs: (Optional) A Python integer giving the number of epochs to train for

    Returns: Nothing, but prints model accuracies during training.
    """
    model = model.to(device=device)  # move the model parameters to CPU/GPU
    for e in range(epochs):
        print("epoch: ",e+1)
        for t, (x, y) in enumerate(loaded_train):
            model.train()  # put model to training mode
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)

            scores = model(x)
            loss = F.cross_entropy(scores, y)

            # Zero out all of the gradients for the variables which the optimizer
            # will update.
            optimizer.zero_grad()

            # This is the backwards pass: compute the gradient of the loss with
            # respect to each  parameter of the model.
            loss.backward()

            # Actually update the parameters of the model using the gradients
            # computed by the backwards pass.
            optimizer.step()

            if t % 100 == 0:
                print('Iteration %d, loss = %.4f' % (t, loss.item()))
                check_accuracy_part(loaded_validation, model)
                print()

Building our neural network

In [ ]:
model = None
optimizer = None

#First architecture #3,32,32
conv1 = nn.Sequential(
    nn.Conv2d(3,512,kernel_size=(3,3),bias=True,padding=1), #512,48,48
    nn.BatchNorm2d(512),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=(2,2))  #Sampling image to half  512,24,24
)
conv2 = nn.Sequential(
    nn.Conv2d(512,128,kernel_size=(3,3),padding=1,bias=True), #128,24,24
    nn.BatchNorm2d(128),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=(2,2))         #128,12,12
)
conv3 = nn.Sequential(
    nn.Conv2d(128,64,kernel_size=(3,3),bias=True,padding=1), #64,12,12
    nn.BatchNorm2d(64),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=(2,2))   #64,6,6
)
conv4 = nn.Sequential(
    nn.Conv2d(64,256,kernel_size=(3,3),bias=True,padding=1), #64,6,6
    nn.BatchNorm2d(256),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=(2,2))   #256,3,3
)
fc = nn.Sequential(
    nn.Flatten(),
    nn.Linear(256*3*3,7),
)
model = nn.Sequential(
    conv1,
    conv2,
    conv3,
    conv4,
    fc
)
learning_rate=0.0001
optimizer = optim.Adam(model.parameters(),lr=learning_rate)
train_part(model, optimizer, epochs=30)

Best performance:

In [ ]:
#Best model
best_model = model
check_accuracy_part(loaded_validation,best_model)

Testing our model with sample images:¶

In [ ]:
# def check_accuracy_test(x,y,model):
#   num_samples = 0
#   num_correct = 0
#   loss = None
#   model.eval()    #turning drop-out/batch norm layer from training to test mode
#   with torch.no_grad():
#     x = x.to(device=device,dtype=dtype)
#     y = y.to(device=device,dtype=torch.long)
#     scores = model(x)
#     _,preds = scores.max(1)
#     return preds

#     expression = {0:"angry",1:"fear",2:"happy",3:"sad",4:"surprise"} #Create a dictionary for mapping accordingly

# transform = T.Compose([T.ToTensor()])
# test_data = dset.ImageFolder("/content/drive/MyDrive/images.colab",transform=transform)
# loaded_test = DataLoader(test_data,batch_size=6,shuffle=False)

# dataiter = iter(loaded_test)   #The iter() function in python represents the iterator similar to c++ iterators
# images, labels = next(dataiter) #The next() method retrieves the object

# random_idx = random.sample(range(0,5),1)[0]

# image = torch.reshape(images[random_idx],(1,3,48,48))
# predicted_idx = check_accuracy_test(image,labels[random_idx],best_model)
# print("Predicted label: ",expression[predicted_idx.item()])
# print("Correct label: ",expression[int(labels[random_idx].numpy())])  #Converting it to numpy from tensor to fetch the label
# image = torch.reshape(image,(3,48,48))
# plt.imshow(np.transpose(image.numpy(), (1, 2, 0)))


In [ ]:
torch.save(best_model, 'model_2.pth')

In [ ]:
model